<a href="https://colab.research.google.com/github/mohammadRahimi1993/BigData/blob/main/CheckGAFineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture 
!pip install ampligraph; 

In [ ]:
%tensorflow_version 1.x
!pip install tensorflow==1.15.2;  
!pip install ypstruct;

TensorFlow 1.x selected.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=9738bcf794be373af3535d357e3eba9f5ac301bb902acc98645025f5b7a48790
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.2.0
    Uninstalling importlib-metadata-4.2.0:
      Successfully uninstalled importlib-metadata-4.2.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is i

In [ ]:
import numpy as np;
import pandas as pd;
import ampligraph;
import numpy as np
import matplotlib.pyplot as plt
from ypstruct import structure
import numpy as np;
import pandas as pd;
import ampligraph;

In [ ]:
def dataPreparing():
#def costFunction():
    from ampligraph.evaluation import train_test_split_no_unseen  
    import requests
    url = 'https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/football_graph.csv'
    open('football_results.csv', 'wb').write(requests.get(url).content)
    df = pd.read_csv("football_results.csv").sort_values("date")
    print(df.isna().sum())
    df = df.dropna()
    df["train"] = df.date < "2014-01-01"
    print(df.train.value_counts())
        # Entities naming
    df["match_id"] = df.index.values.astype(str)
    df["match_id"] =  "Match" + df.match_id
    df["city_id"] = "City" + df.city.str.title().str.replace(" ", "")
    df["country_id"] = "Country" + df.country.str.title().str.replace(" ", "")
    df["home_team_id"] = "Team" + df.home_team.str.title().str.replace(" ", "")
    df["away_team_id"] = "Team" + df.away_team.str.title().str.replace(" ", "")
    df["tournament_id"] = "Tournament" + df.tournament.str.title().str.replace(" ", "")
    df["neutral"] = df.neutral.astype(str)
    
    triples = []
    for _, row in df[df["train"]].iterrows():
        # Home and away information
        home_team = (row["home_team_id"], "isHomeTeamIn", row["match_id"])
        away_team = (row["away_team_id"], "isAwayTeamIn", row["match_id"])
    
        # Match results
        if row["home_score"] > row["away_score"]:
            score_home = (row["home_team_id"], "winnerOf", row["match_id"])
            score_away = (row["away_team_id"], "loserOf", row["match_id"])
        elif row["home_score"] < row["away_score"]:
            score_away = (row["away_team_id"], "winnerOf", row["match_id"])
            score_home = (row["home_team_id"], "loserOf", row["match_id"])
        else:
            score_home = (row["home_team_id"], "draws", row["match_id"])
            score_away = (row["away_team_id"], "draws", row["match_id"])
        home_score = (row["match_id"], "homeScores", np.clip(int(row["home_score"]), 0, 5))
        away_score = (row["match_id"], "awayScores", np.clip(int(row["away_score"]), 0, 5))
    
        # Match characteristics
        tournament = (row["match_id"], "inTournament", row["tournament_id"])
        city = (row["match_id"], "inCity", row["city_id"])
        country = (row["match_id"], "inCountry", row["country_id"])
        neutral = (row["match_id"], "isNeutral", row["neutral"])
        year = (row["match_id"], "atYear", row["date"][:4])
    
        triples.extend((home_team, away_team, score_home, score_away, 
                        tournament, city, country, neutral, year, home_score, away_score))
    triples_df = pd.DataFrame(triples, columns=["subject", "predicate", "object"])
    triples_df[(triples_df.subject=="Match3129") | (triples_df.object=="Match3129")]
    
    X_train, X_valid = train_test_split_no_unseen(np.array(triples), test_size=10000)
    print('Train set size: ', X_train.shape)
    print('Test set size: ', X_valid.shape)
    return X_train, X_valid

In [ ]:
X_train, X_valid=dataPreparing()

date          0
home_team     0
away_team     0
home_score    2
away_score    2
tournament    0
city          0
country       0
neutral       0
dtype: int64
True     35714
False     5057
Name: train, dtype: int64
Train set size:  (382854, 3)
Test set size:  (10000, 3)


In [ ]:
def costFunction(X, X_train, X_valid):
    from ampligraph.latent_features import ComplEx
    model = ComplEx(batches_count=int(X[0]),
                    epochs=10,
                    k=int(X[2]),
                    eta=int(X[1]),
                    optimizer='adam', 
                    optimizer_params={'lr':1e-4},
                    loss='multiclass_nll',
                    regularizer='LP', 
                    regularizer_params={'p':3, 'lambda':1e-5}, 
                    seed=0, 
                    verbose=True)
    import tensorflow as tf
    tf.logging.set_verbosity(tf.logging.ERROR)
    model.fit(X_train)
    #model.fit(X_train[:10000,:10000])
    filter_triples = np.concatenate((X_train, X_valid))
    from ampligraph.evaluation import evaluate_performance
    #ranks = evaluate_performance(X_valid[:100,:200],
    #                             model=model, 
    #                             filter_triples=filter_triples,
    #                             use_default_protocol=True,
    #                             verbose=True)
    ranks = evaluate_performance(X_valid,
                                 model=model, 
                                 filter_triples=filter_triples,
                                 use_default_protocol=True,
                                 verbose=True)
    
    from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

    mr = mr_score(ranks)
    mrr = mrr_score(ranks)

    print("MRR: %.2f" % (mrr))
    print("MR: %.2f" % (mr))

    hits_10 = hits_at_n_score(ranks, n=10)
    print("Hits@10: %.2f" % (hits_10))
    hits_3 = hits_at_n_score(ranks, n=3)
    print("Hits@3: %.2f" % (hits_3))
    hits_1 = hits_at_n_score(ranks, n=1)
    print("Hits@1: %.2f" % (hits_1))
    return float(mrr)


In [ ]:
def crossover(p1, p2, gamma=0.1):
    c1 = p1.deepcopy()
    c2 = p1.deepcopy()
    alpha = np.random.uniform(-gamma, 1+gamma, *c1.position.shape)
    print('alpha:', alpha)
    c1.position = alpha*p1.position + (1-alpha)*p2.position
    print('alpha*p1.position:', alpha*p1.position)
    print('(1-alpha)*p2.position',(1-alpha)*p2.position)
    c2.position = alpha*p2.position + (1-alpha)*p1.position
    print('alpha*p1.position:', alpha*p2.position)
    print('(1-alpha)*p2.position',(1-alpha)*p1.position)
    return c1, c2

def mutate(x, mu, sigma):
    y = x.deepcopy()
    flag = np.random.rand(*x.position.shape) <= mu
    ind = np.argwhere(flag)
    y.position[ind] += sigma*np.random.randn(*ind.shape)
    return y

def apply_bound(x, varmin, varmax):
    x.position = np.maximum(x.position, varmin)
    x.position = np.minimum(x.position, varmax)

def roulette_wheel_selection(p):
    c = np.cumsum(p)
    r = sum(p)*np.random.rand()
    ind = np.argwhere(r <= c)
    return ind[0][0]


In [ ]:
# Problem Definition
problem = structure()
problem.costFunc = costFunction
problem.nvar = 3
#X[0]:Batch ,X[1]:eta ,X[2]:Dimention Of Embbeding
problem.varmin = [30,20,30]
problem.varmax = [100,70,100]

# GA Parameters
params = structure()
params.maxit = 15
params.npop = 3
params.beta = 1
params.pc = 1
params.gamma = 0.1
params.mu = 0.9
params.sigma = 20

In [ ]:
# Problem Information
# costFunction = problem.costFunction
popc = []
nvar = problem.nvar
varmin = problem.varmin
varmax = problem.varmax

# Parameters
maxit = params.maxit
npop = params.npop
beta = params.beta
pc = params.pc
nc = int(np.round(pc*npop/2)*2)
gamma = params.gamma
mu = params.mu
sigma = params.sigma

# Empty Individual Template
empty_individual = structure()
empty_individual.position = None
empty_individual.cost = None

# Best Solution Ever Found
bestsol = empty_individual.deepcopy()
bestsol.cost = 0

pop = empty_individual.repeat(npop)
bestSolution = empty_individual.repeat(maxit)

# Initialize Population
for i in range(npop):
    pop[i].position = np.random.uniform(varmin, varmax, nvar) 
    print(pop[i].position)
for i in range(npop):
    print(pop[i].position)
    pop[i].cost = costFunction(pop[i].position, X_train, X_valid)
    print(pop[i].cost)
    if pop[i].cost > bestsol.cost:
        bestsol = pop[i].deepcopy()
    popc.append(pop[i])

# Best Cost of Iterations
bestposition = np.empty(maxit)
bestcost = np.empty(maxit)

# Main Loop
for it in range(maxit):
    costs = np.array([x.cost for x in pop])
    avg_cost = np.mean(costs)
    if avg_cost != 0:
        costs = costs/avg_cost
    probs = np.exp(-beta*costs)

    for _ in range(nc//2):

        # Select Parents
        #q = np.random.permutation(npop)
        #p1 = pop[q[0]]
        #p2 = pop[q[1]]

        # Perform Roulette Wheel Selection
        p1 = pop[roulette_wheel_selection(probs)]
        p2 = pop[roulette_wheel_selection(probs)]
        print('P1:', p1)
        print('P2:',p2)
  
        # Perform Crossover
        # c1, c2 = crossover(p1, p2, gamma)
        gamma = 0.01
        c1 = p1.deepcopy()
        c2 = p2.deepcopy()
        alpha = np.random.uniform(-gamma, 1+gamma, *c1.position.shape)
        c1.position = alpha*p1.position + (1-alpha)*p2.position
        c2.position = alpha*p2.position + (1-alpha)*p1.position
        apply_bound(c1, varmin, varmax)
        apply_bound(c2, varmin, varmax)

        # Evaluate First Offspring
        flag = False
        for element in popc:
          if (np.array_equal(element.position , c1.position)):
            print("Dynamic Programming")
            c1.cost = element.cost
            flag = True
            break;
        if (flag == False):
          print(c1.position) 
          c1.cost = costFunction(c1.position, X_train, X_valid)
          if c1.cost > bestsol.cost:
            bestsol = c1.deepcopy()

        # Evaluate Second Offspring
        flag = False
        for element in popc:
          if (np.array_equal(element.position , c2.position)):
            print("Dynamic Programming")
            c2.cost = element.cost
            flag = True
            break;
        if (flag == False):
          print(c2.position) 
          c2.cost = costFunction(c2.position, X_train, X_valid)
          if c2.cost > bestsol.cost:
            bestsol = c2.deepcopy()

        print('crossover c1:',c1)
        print('crossover c2:',c2) 

        # Perform Mutation
        c3 = c1.deepcopy()
        flag = np.random.rand(*c1.position.shape) <= mu
        ind = np.argwhere(flag)
        c3.position[ind] += sigma*np.random.randn(*ind.shape)
        #c1 = mutate(c1, mu, sigma)
        c4 = c2.deepcopy()
        flag = np.random.rand(*c2.position.shape) <= mu
        ind = np.argwhere(flag)
        c4.position[ind] += sigma*np.random.randn(*ind.shape)
        #c2 = mutate(c2, mu, sigma)
              
        # Apply Bounds
        apply_bound(c3, varmin, varmax)
        apply_bound(c4, varmin, varmax)

        # Evaluate Thirty Offspring
        flag = False
        for element in popc:
          if (np.array_equal(element.position , c3.position)):
            print("Dynamic Programming")
            c3.cost = element.cost
            flag = True
            break;
        if (flag == False):
          print(c3.position) 
          c3.cost = costFunction(c3.position, X_train, X_valid)
          if c3.cost > bestsol.cost:
            bestsol = c3.deepcopy()

        # Evaluate Fortith Offspring
        flag = False
        for element in popc:
          if (np.array_equal(element.position , c4.position)):
            print("Dynamic Programming")
            c4.cost = element.cost
            flag = True
            break;
        if (flag == False):
          print(c4.position) 
          c4.cost = costFunction(c4.position, X_train, X_valid)
          if c4.cost > bestsol.cost:
            bestsol = c4.deepcopy()

        print('mutate c3:',c3)
        print('mutate c4:',c4) 
        # Add Offsprings to popc

        popc.append(c1)
        popc.append(c2)
        popc.append(c3)
        popc.append(c4)
        
        # Check pop list for insert c1,c2,c3,c4
        flag = False
        for element in pop:
          flag = np.array_equal(element.position, c1.position)
          if flag == True:
            break
        if flag == False:
          pop.append(c1)
        
        flag = False
        for element in pop:
          flag = np.array_equal(element.position, c2.position)
          if flag == True:
            break
        if flag == False:
          pop.append(c2)

        flag = False
        for element in pop:
          flag = np.array_equal(element.position, c3.position)
          if flag == True:
            break
        if flag == False:
          pop.append(c3)

        flag = False
        for element in pop:
          flag = np.array_equal(element.position, c4.position)
          if flag == True:
            break
        if flag == False:
          pop.append(c4)


    # Merge, Sort and Select
    # pop += popc
    pop = sorted(pop, key=lambda x: x.cost, reverse=True)
    pop = pop[0:npop]
    print('pop:' , pop)
    
    # Store Best Cost
    bestcost[it] = bestsol.cost

    bestSolution[it].cost = bestsol.cost
    bestSolution[it].position = bestsol.position
                               
    # Show Iteration Information
    print("Iteration {}: Best Cost = {}".format(it, bestcost[it]))


# Output
out = structure()
out.pop = pop
out.bestsol = bestsol
out.bestcost = bestcost
print(out.pop)
print(out.bestsol)
print(out.bestcost)

[46.3480393  66.07771834 57.77267583]
[70.67345668 56.2462999  53.0812863 ]
[51.47704488 53.30424993 73.61647217]
[46.3480393  66.07771834 57.77267583]


Average ComplEx Loss:   4.199490: 100%|██████████| 10/10 [00:52<00:00,  5.21s/epoch]


WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 10000/10000 [02:44<00:00, 60.62it/s]


MRR: 0.10
MR: 10490.63
Hits@10: 0.17
Hits@3: 0.12
Hits@1: 0.07
0.10480533983728282
[70.67345668 56.2462999  53.0812863 ]


...


100%|██████████| 10000/10000 [03:06<00:00, 53.55it/s]


MRR: 0.17
MR: 8830.14
Hits@10: 0.27
Hits@3: 0.20
Hits@1: 0.12
[100.          20.          83.65418194]


Average ComplEx Loss:   2.997627: 100%|██████████| 10/10 [00:26<00:00,  2.64s/epoch]

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.



 64%|██████▍   | 6441/10000 [01:51<01:01, 57.75it/s]

In [2]:
df = pd.DataFrame({"position": [bestSolution[0].position] , "cost": [bestSolution[0].cost]})
for i in range(params.maxit):
    if(i==0):
        continue;
    df.loc[len(df.index)] = [bestSolution[i].position , bestSolution[i].cost] 
df.to_excel('result.xlsx')
df